In [ ]:
import os,json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize
class CVManager:
    def __init__(self):
     
        self.CVAdded=None
        #self.database=databaseHandler()
        self.CVList=[]
        self.cvsFile="documentMatrix.csv"
        self.CVFileName=[]
        self.fileNamesWithPath=[]
        self.cvPostList=[]
        self.CVTextColl=[]
        self.CVTextCollTest=[]
        self.documentMatrix=[]
        self.normalizedFeatureSet=[]
        self.vocabulary={}
        self.documentMatrixTest=[]
        self.vocabularyTest={}
        self.clusterInfo={}
        self.clustersToForm=None
        self.kMeans=None
        self.impFeaturesIndexList=[]
        self.topWords=None
        self.scoreList=[]
        self.clusterWiseTopWordList=[]
        self.overAllWeight=[]
    def list_CVs(self,rootPath):
          for root, dirs, files in os.walk(rootPath):
            for name in files:
                self.CVFileName.append(name)
                self.fileNamesWithPath.append(os.path.join(root, name))
                self.cvPostList.append(os.path.basename(os.path.dirname(os.path.join(root,name))))
               
    def collectCV(self):
            for cvFilePath,cvFileName,cvPost in zip(self.fileNamesWithPath,self.CVFileName,self.cvPostList):
                try:
                    newCV=CV(cvFileName,cvFilePath,cvPost)
                    self.CVList.append(newCV)
                except Exception as e:
                    print(cvFileName)
                    print("in collection of CV \t"+str(e))
            
    def collectCVText(self):
        self.CVTextColl=[]
        self.CVTextCollTest=[]
        for cv in self.CVList:
            self.CVTextColl.append(cv.cleanText) 
            self.CVTextCollTest.append(cv.stemedText) 
    def findDocumentMatrix(self,minFrequency,vocab):   
#         try:
                self.documentMatrix=[]
                self.vocabulary=[]
                self.documentMatrixTest=[]
                self.vocabularyTest=[]
                self.collectCVText()
                self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency,vocab)
                df = pd.DataFrame(self.documentMatrix.toarray())
                df.to_csv(self.cvsFile)
                
                self.documentMatrixTest,self.vocabularyTest=NLTKHelper.findDocumentMatrix(self.CVTextCollTest,minFrequency,vocab)
#         except Exception as e:
#             print("in reading/writing of saved files \t"+str(e))
    def normalizeFeatureVector(self):
        import numpy as np
        self.normalizedFeatureSet=normalize(self.documentMatrix.toarray().astype('float64'))
        for cv,cvNum in zip(manager.CVList,range(0,len(manager.CVFileName)-1)):
            for featureRow in range(0,len(self.vocabularyTest)-1):
                cv.featureVector.append(self.normalizedFeatureSet[cvNum][featureRow])
                
    def getClusterNumber(self):
        self.clusterInfo=Clustering.findclusterInfo(self.normalizedFeatureSet)
        self.makeGraph()
        maxValue=max(self.clusterInfo.values())
        max_keys = [k for k, v in self.clusterInfo.items() if v == maxValue]
        if(len(max_keys)==1):
            for x in max_keys:
                self.clustersToForm=x
        else:
            print("2 keys,confusion")
        
    def makeGraph(self):
        lists = sorted(self.clusterInfo.items()) # sorted by key, return a list of tuples
        x, y = zip(*lists) # unpack a list of pairs into two tuples
        plt.plot(x, y)
        plt.show()
        
    def clusterData(self):
        self.getClusterNumber()
        self.kMeans=Clustering.clusterData(self.clustersToForm,self.normalizedFeatureSet)
        self.getTopWords()
        self.calculateCVScore()
        
    def getTopWords(self):
        x=[]
        for key,values in manager.vocabulary.items():
            x.append(values)        
        self.topWords=refiefFAlgo()
        self.topWords.reliefF(self.kMeans.cluster_centers_,np.asarray(x),k=self.clustersToForm-1)
        for clusterNo,impFeaturesRow in enumerate(self.kMeans.cluster_centers_):
            WordList={}
            for indexNo in self.topWords.wordIndex[clusterNo]:
                WordList.update({list(manager.vocabulary.keys())[list(manager.vocabulary.values()).index(indexNo)]:manager.topWords.scoreList[clusterNo][indexNo]})
            self.clusterWiseTopWordList.append(WordList)
        self.getOverallWeightOfRelevantWords()
    def getOverallWeightOfRelevantWords(self):
        import numpy as np
        self.overAllWeight=np.sum([x for x in self.topWords.scoreList],axis=0)
    def calculateCVScore(self):
        #self.scoreList=CBRAlgo.calculateCVScore(self.documentMatrix,self.topWords)
        self.scoreList=CBRAlgo.calculateCVScore(self.documentMatrix,self.overAllWeight)
        for cv,cvScore in zip(self.CVList,self.scoreList):
            cv.score=cvScore
